In [1]:
import numpy as np
import pandas as pd
from django.forms import model_to_dict

from vvs_map.models import VVSTransport

In [2]:
qs_transport = VVSTransport.objects.all()
qs_journey = VVSJourney.objects.all()
qs_data = VVSData.objects.all()

In [3]:
# Get all object from QuerySet to a DataFrame
# Transport
objs = [ model_to_dict(obj) for obj in qs_transport] 
df_transport = pd.DataFrame(objs)
# Journey
objs = [ model_to_dict(obj) for obj in qs_journey] 
df_journey = pd.DataFrame(objs)
# Data
objs = [ model_to_dict(obj) for obj in qs_data] 
df_data = pd.DataFrame(objs)

In [4]:
# VVSTransport.journey_id -> int docu defined as: external journey id,
# does it have lot of duplicates? Can we use it for something
# let's serch for a specified journey_id
qs = VVSTransport.objects.filter(journey_id=41)
objs = [model_to_dict(obj) for obj in qs] 
df_journey_id = pd.DataFrame(objs)
df_journey_id
# we can see different direction and line texts for the same journey_id 41
# so it is really something 'internal' id

,direction_text,id,journey_id,line_text,mod_code,operator,product_id
0,Stammheim,123,41,Stadtbahn U15,3,,PTTYPE-3
1,Hedelfingen,185,41,Stadtbahn U9,3,,PTTYPE-3
2,Rohr Mitte,340,41,Bus 82,5,,PTTYPE-5
3,SchÃ¶nberg,407,41,Bus 71,5,,PTTYPE-5
4,Kirchheim (T) Bahnhof/ZOB,965,41,Bus 168,5,,PTTYPE-5
5,Echterdingen Flughafen/Messe,976,41,Bus 79,5,,PTTYPE-5
6,Winterbach RemsbrÃ¼cke,1502,41,Bus 106,5,,PTTYPE-5
7,Eltingen Leo-Center,1665,41,Bus 630,5,,PTTYPE-5
8,IndustriestraÃe,1744,41,Bus 80,5,,PTTYPE-5
9,Europaplatz,1758,41,Bus 72,5,,PTTYPE-5


In [5]:
# do we have duplicated vvs_transport(id) in df_journey?
df_journey['vvs_transport'].value_counts().head()
# -- yes we have -> the same transport on the same day multiple times

15875    28
13838    19
15567    17
14448    15
15057    15
Name: vvs_transport, dtype: int64

In [6]:
VVSTransport.objects.get(id=1)

<VVSTransport: Linie Bus 828, Richtung TÃ¼bingen Hauptbahnhof, Journey_id 33966>

In [7]:
# try to avoid big merged tables
df_transport_bus82 = df_transport[df_transport['line_text'] =='Bus 82']

In [8]:
# select row from journey table with Bus 82
df_journey_bus82 = df_journey[df_journey['vvs_transport'].isin(df_transport_bus82.id)]

In [9]:
df_data_bus82 = df_data[df_data['vvs_journey'].isin(df_journey_bus82.id)]

In [10]:
# add direction text field to the end of this table
df_direction = pd.DataFrame(df_transport_bus82, columns=['id','direction_text'])

# merge with jorney 
df_direction_merged = pd.merge(df_journey_bus82, df_direction, left_on='vvs_transport', right_on='id')

# kep just the necesary columns
df_direction_merged_min = pd.DataFrame(df_direction_merged, columns=['id_x','direction_text'])

In [11]:
# merge data and direction frames
df_data_bus82_direction = pd.merge(df_direction_merged_min, df_data_bus82, left_on='id_x', right_on='vvs_journey')

In [12]:
# as the direction changes, the journey_id is changing too
pd.DataFrame(df_data_bus82_direction, columns=['direction_text','vvs_journey','timestamp','next_stop'])

,direction_text,vvs_journey,timestamp,next_stop
0,Rohr Mitte,340,2016-02-01 09:08:42+00:00,5002595#0#3
1,Rohr Mitte,340,2016-02-01 09:09:10+00:00,5006004#2#3
2,Rohr Mitte,340,2016-02-01 09:10:11+00:00,5006004#2#3
3,Rohr Mitte,340,2016-02-01 09:10:39+00:00,5006004#2#3
4,Rohr Mitte,340,2016-02-01 09:11:07+00:00,5006004#2#3
5,Rohr Mitte,340,2016-02-01 09:11:35+00:00,5006004#2#3
6,Rohr Mitte,340,2016-02-01 09:12:10+00:00,5006012#1#3
7,Rohr Mitte,340,2016-02-01 09:12:37+00:00,5006012#1#3
8,Rohr Mitte,340,2016-02-01 09:13:11+00:00,5006002#7#6
9,Rohr Mitte,340,2016-02-01 09:13:39+00:00,5006002#7#6
